Run selection
===========

Anastasia has summarized the standalone runs of the TRD in 2021 in the Jira ticket [\[TRD-112\]](https://alice.its.cern.ch/jira/browse/TRD-112).

From this ticket, I picked run 504428 with the following comment:
```
504419, 100 hz, 42212, calrate 1; 2.3GB/s, 100000 events
504425, 1 hz, 42212, calrate 1; 151MB/s, 1000 events
504428, 10 khz, 42212, calrate 1; 151MB/s, 49944 events ; accepted rate 5 k
```

In [24]:
run = 521651
filepattern = f"/alice/data/2022/LHC22m/{run}/raw/1840"

outdir = f"/eos/user/j/jpotgiet/data/noise/{run}"

# Path(outdir).mkdir(parents=True, exist_ok=True)
%cd $outdir

/eos/home-i01/j/jpotgiet/data/noise/521651


In [25]:
from IPython.core.magic import register_cell_magic
import string
import re
import argparse
import pathlib

class NamespaceFormatter(string.Formatter):
   def __init__(self, namespace={}):
       string.Formatter.__init__(self)
       self.namespace = namespace

   def get_value(self, key, args, kwds):
       if isinstance(key, str):
           try:
               # Check explicitly passed arguments first
               return kwds[key]
           except KeyError:
               return self.namespace[key]
       else:
           string.Formatter.get_value(key, args, kwds)

            
@register_cell_magic
def genfile(line, cell):
    """ Generate file 
    
    This magic is an extension of the %%writefile magic. The additional functionality 
    includes:
      - variable expansion (similar to string.format and f-strings)
      - set file permisssions with -m | --mode flag"""

    # parse the command line arguments
    parser = argparse.ArgumentParser(description='Generate file magic')
    parser.add_argument('outfile', type=str, help='output file')
    parser.add_argument('-m', '--mode', help='file permissions')
    args = parser.parse_args(line.split())


    # format the content
    fmt = NamespaceFormatter(globals())
    content = fmt.format(cell)
    content = content.replace("\n    ", "\n\t")

    # TODO: this should check if the file content changed to
    # prevent updating the timestamp if nothing changed.
    # This will help with Makefile dependencies.
    
    # generate the file
    filename = pathlib.Path(args.outfile)
    with open(filename, "w") as out:
            out.write(content)
    if args.mode is not None:
        filename.chmod(int(args.mode,8))

    print(f"Generated file '{filename}'")

    return 

In [26]:
info = %system alien-token-info
print(info)

import alienpy.alien as alien

alien.setup_logging()
try:
    j = alien.AliEn()
#     j.ProcessMsg('whoami -v')

except SystemExit:
    print("ERROR initializing AliEn session")
    print("Make sure you have a valid token by opening a SWAN terminal and running `alien-token-init`")

['DN >>> /C=ch/O=AliEn2/CN=Users/CN=jpotgiet/OU=jpotgiet', 'ISSUER >>> /C=ch/O=AliEn2/CN=AliEn CA', 'BEGIN >>> 2022-09-08 09:31:04', 'EXPIRE >>> 2022-10-09 11:31:04']


In [27]:
import json
import collections
from pathlib import Path
from pprint import pprint

AlienFileInfo = collections.namedtuple('AlienFileInfo', ['path','size'])
results = j.run(f'ls -l {filepattern}').ansdict['results']
fileinfo = [AlienFileInfo(Path(x['path']), int(x['size'])) for x in results]
# print(fileinfo)

# OPTIONAL: restrict results
fileinfo = fileinfo[1:4]

In [28]:
totalsize = sum([x.size for x in fileinfo])
print(f"Using {len(fileinfo)} files with {totalsize / float(1<<30):.3f} GiB")
print()

# for x in fileinfo: print(f"{x.size:15d}  {x.path}")
    
with open(Path(outdir)/"files.txt", "w") as f:
    for x in fileinfo:
        #print(f"{x.size:15d}  {x.path}")
        f.write(str(x.path))
        
print(fileinfo)

Using 3 files with 5.601 GiB

[AlienFileInfo(path=PosixPath('/alice/data/2022/LHC22m/521651/raw/1840/o2_ctf_run00521651_orbit0332422920_tf0000000002_epn148.root'), size=2006467400), AlienFileInfo(path=PosixPath('/alice/data/2022/LHC22m/521651/raw/1840/o2_ctf_run00521651_orbit0332423048_tf0000000003_epn115.root'), size=2002870261), AlienFileInfo(path=PosixPath('/alice/data/2022/LHC22m/521651/raw/1840/o2_ctf_run00521651_orbit0332423176_tf0000000004_epn185.root'), size=2005103872)]


### Copying files

It's straightforward to download a file with `alien.py`

In [29]:
outdir = pathlib.Path(outdir)

for f in [ x.path for x in fileinfo]:
    if not (outdir / f.name).exists():
        j.ProcessMsg(f"cp {f} file:{outdir / f.name}")
    
%system ls -lh $outdir

jobID: 1/1 >>> Start
jobID: 1/1 >>> STATUS OK >>> SPEED 5.58 MiB/s
Succesful jobs (1st try): 1/1
jobID: 1/1 >>> Start
jobID: 1/1 >>> STATUS OK >>> SPEED 6.16 MiB/s
Succesful jobs (1st try): 1/1
jobID: 1/1 >>> Start
jobID: 1/1 >>> STATUS OK >>> SPEED 10.95 MiB/s
Succesful jobs (1st try): 1/1


['total 5.7G',
 '-rw-r--r--. 1 jpotgiet 1395  297 Sep  8 15:14 files.txt',
 '-rw-r--r--. 1 jpotgiet 1395 1.9G Sep  8 15:20 o2_ctf_run00521651_orbit0332422920_tf0000000002_epn148.root',
 '-rw-r--r--. 1 jpotgiet 1395 1.9G Sep  8 15:25 o2_ctf_run00521651_orbit0332423048_tf0000000003_epn115.root',
 '-rw-r--r--. 1 jpotgiet 1395 1.9G Sep  8 15:28 o2_ctf_run00521651_orbit0332423176_tf0000000004_epn185.root']

## Convert timeframe


In [30]:
%%genfile -m 0755 convert.sh
#!/bin/sh

eval $(alienv printenv VO_ALICE@O2::nightly-20220728-1)

o2-ctf-reader-workflow --onlyDet TRD --ctf-input files.txt \
| o2-trd-digittracklet-writer \
--run --batch

Generated file 'convert.sh'


In [31]:
%%genfile Makefile

conv: convert.sh files.txt
    ./convert.sh

clean:
    rm -f trdtracklets.root trddigits.root

Generated file 'Makefile'


In [32]:
%system ls -l
# %system hexdump -C Makefile

['total 5873480',
 '-rwxr-xr-x. 1 jpotgiet 1395        175 Sep  8 15:28 convert.sh',
 '-rw-r--r--. 1 jpotgiet 1395        297 Sep  8 15:14 files.txt',
 '-rw-r--r--. 1 jpotgiet 1395         90 Sep  8 15:29 Makefile',
 '-rw-r--r--. 1 jpotgiet 1395 2006467400 Sep  8 15:20 o2_ctf_run00521651_orbit0332422920_tf0000000002_epn148.root',
 '-rw-r--r--. 1 jpotgiet 1395 2002870261 Sep  8 15:25 o2_ctf_run00521651_orbit0332423048_tf0000000003_epn115.root',
 '-rw-r--r--. 1 jpotgiet 1395 2005103872 Sep  8 15:28 o2_ctf_run00521651_orbit0332423176_tf0000000004_epn185.root']